In [1]:
import sqlite3

In [7]:
# Connect to DB and create a cursor
db_connection = sqlite3.connect('bookmarks.db')
cur = db_connection.cursor()

# Create a table
cur.execute('''
    CREATE TABLE IF NOT EXISTS links (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        URL TEXT
    )
''')

In [11]:
cur.execute('''
            INSERT INTO links (url)
            VALUES (?)
            ''', ('https://www.londonambulance.nhs.uk/working-for-us/volunteering-with-us/emergency-responders/',))
db_connection.commit()

In [15]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('links',)]

In [ ]:
cursor.execute('''
    INSERT INTO users (name, age) 
    VALUES (?, ?)
''', ('Alice', 30))